<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#READ-ME" data-toc-modified-id="READ-ME-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>READ ME</a></span></li><li><span><a href="#Load-all-contracts" data-toc-modified-id="Load-all-contracts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load all contracts</a></span></li><li><span><a href="#Load-symbols-lut" data-toc-modified-id="Load-symbols-lut-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load symbols lut</a></span></li><li><span><a href="#Get-expiration-dates" data-toc-modified-id="Get-expiration-dates-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get expiration dates</a></span><ul class="toc-item"><li><span><a href="#OLD-RULE:-12-business-days-before-the-start-of-the-symbol-month" data-toc-modified-id="OLD-RULE:-12-business-days-before-the-start-of-the-symbol-month-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>OLD RULE: 12 business days before the start of the symbol month</a></span></li><li><span><a href="#NEW-RULE:-last-business-day-of-the-second-month-previous-to-the-symbol" data-toc-modified-id="NEW-RULE:-last-business-day-of-the-second-month-previous-to-the-symbol-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>NEW RULE: last business day of the second month previous to the symbol</a></span></li><li><span><a href="#Concatenate-rules" data-toc-modified-id="Concatenate-rules-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Concatenate rules</a></span></li><li><span><a href="#Save" data-toc-modified-id="Save-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Save</a></span></li></ul></li><li><span><a href="#Sanity-check" data-toc-modified-id="Sanity-check-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Sanity check</a></span></li></ul></div>

# READ ME

This notebook creates a table with expiration dates for each contract 


In [1]:
% matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import datetime
import numpy as np
import os

In [3]:
from cqcplot import *

/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load symbols lut

In [4]:
symbols_lut = pd.read_csv('./../month_code_lut.csv')

In [5]:
symbols_lut

,month,code
0,1,F
1,2,G
2,3,H
3,4,J
4,5,K
5,6,M
6,7,N
7,8,Q
8,9,U
9,10,V


# Get expiration dates

In [6]:
start_dates = pd.DataFrame({'Date':pd.date_range(start=datetime.datetime(2009,1,1), end = datetime.datetime(2020,12,31))})

In [7]:
start_dates['month'] = start_dates['Date'].dt.month
start_dates['day'] = start_dates['Date'].dt.day
start_dates['dow'] = start_dates['Date'].dt.weekday

start_dates['year_symbol'] = start_dates['Date'].dt.year.astype(str).str[-2:]

## OLD RULE: 12 business days before the start of the symbol month

In [8]:
old_rule = start_dates.copy().loc[start_dates['Date']<datetime.datetime(2016,2,1)]

In [9]:
old_rule['counts']=1
old_rule = old_rule.sort_values('Date',ascending=False)\
          .query('dow!=6 & dow!=5')

In [10]:
old_rule['position'] = old_rule.groupby(['month', 'year_symbol'], sort=False)['counts'].cumsum()

In [11]:
old_rule = old_rule.sort_values('Date').query('position==12')

In [12]:
# clean up
old_rule.rename(columns={'Date':'expiration_date'}, inplace=True)
old_rule = old_rule[['expiration_date']]

In [13]:
old_rule['contract_delivery_date'] = old_rule['expiration_date'].dt.date + pd.DateOffset(months=1)

In [14]:
old_rule['month'] = old_rule['contract_delivery_date'].dt.month
old_rule['year_symbol'] = old_rule['contract_delivery_date'].dt.year.astype(str).str[-2:]

In [15]:
old_rule = pd.merge(old_rule, symbols_lut, on='month', how='left')

In [16]:
old_rule['Symbol_end'] = 'CO' + old_rule['code'] + old_rule['year_symbol']

In [17]:
# clean up
old_rule = old_rule[['expiration_date', 'Symbol_end']]

## NEW RULE: last business day of the second month previous to the symbol

In [18]:
new_rule = start_dates.copy()\
                      .loc[start_dates['Date']>=datetime.datetime(2016,1,29)]\
                      .query('dow!=6 & dow!=5')\
                      .groupby(['month', 'year_symbol'])\
                      .last()\
                      .reset_index()\
                      .rename(columns={'Date':'expiration_date'})

In [19]:
new_rule = new_rule[['expiration_date']]
new_rule['contract_delivery_date'] = new_rule['expiration_date'].dt.date + pd.DateOffset(months=2)

In [20]:
new_rule['month'] = new_rule['contract_delivery_date'].dt.month
new_rule['year_symbol'] = new_rule['contract_delivery_date'].dt.year.astype(str).str[-2:]

In [21]:
new_rule = pd.merge(new_rule, symbols_lut, on='month', how='left')

In [22]:
new_rule['Symbol_end'] = 'CO' + new_rule['code'] + new_rule['year_symbol']

In [23]:
# clean up
new_rule = new_rule[['expiration_date', 'Symbol_end']]

## Concatenate rules

In [24]:
expiration_dates = pd.concat([old_rule, new_rule]).sort_values('expiration_date')

In [25]:
expiration_dates['Symbol_start'] = expiration_dates['Symbol_end'].shift(-1)
expiration_dates=expiration_dates.dropna()

In [26]:
expiration_dates['expiration'] = expiration_dates['expiration_date']+datetime.timedelta(hours=23)

## Save

In [27]:
expiration_dates.to_csv('./../expiration_dates.csv')